# Hotel

In [33]:
import pandas as pd
import nltk
import xmltodict
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString

import warnings
warnings.filterwarnings('ignore')

In [22]:
%run ../funcoes.py

Loading functions...


In [23]:
domain = "hotel"
dfTest = pd.read_csv('../07_Keras/data/domain_hotel_IOB_test.csv', sep='\t', quoting=3, keep_default_na=False)
dfTrain = pd.read_csv('../07_Keras/data/domain_hotel_IOB_train.csv', sep='\t', quoting=3, keep_default_na=False)

In [26]:
# carregando tipos
tiposAll = pd.read_csv('../data/tipologia/categorias_iacs.csv', sep=',', quoting=0, keep_default_na=False)
tipos = tiposAll.loc[tiposAll['domain'] == domain]

In [4]:
f = open("../07_Keras/data/aspectsTest.xml","r")
xml = f.read()
aspectsTest = xmltodict.parse(xml, process_namespaces=True)

In [5]:
def result2aspecList(y_pred, X_test):
    aspectsList = []
    aspect = []
    stopwords = nltk.corpus.stopwords.words('portuguese')
    for sent, sent_pred in zip(X_test, y_pred):
        #print(len(sent),len(sent_pred))
        for token, tag in zip(sent, sent_pred):
            if tag != 'O':
                #print(tag, token)
                #print(tag, token['word.lower()'])
                if tag == 'B-asp' and aspect:
                    auxAspect = " ".join(aspect).lower()
                    if (auxAspect not in stopwords and len(auxAspect) > 1):
                        aspectsList.append(auxAspect)
                    aspect = []
                if token not in [',',':']:
                    aspect.append(token.lower())

    aspectsList = list(set(aspectsList))
    aspectsList.sort()    
    return aspectsList

In [6]:
tag_scheme = [
'B-asp',
'I-asp'
]

In [7]:
contSents = 0
for count, row in dfTrain.iterrows():
    if (row[0] != "" and count > 0):
        contSents = contSents + 1
contSents = contSents + 1

In [8]:
contTrain = round(contSents * 0.9)
print(contTrain)

614


In [9]:
training = {'sentences': [], 'tags' : []}
validation = {'sentences': [], 'tags' :[]}
sent = []
tags = []
contSents = 0
for count, row in dfTrain.iterrows():
    if (row[0] != "" and count > 0):
        if contSents < contTrain:
            training['sentences'].append(sent)
            training['tags'].append(tags)
        else:
            validation['sentences'].append(sent)
            validation['tags'].append(tags)

        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
        
validation['sentences'].append(sent)
validation['tags'].append(tags)

In [10]:
len(training['sentences'])

614

In [11]:
len(validation['sentences'])

68

In [12]:
test = {'sentences': [], 'tags' : []}
sent = []
tags = []
contSents = 0
for count, row in dfTest.iterrows():
    if (row[0] != "" and count > 0):
        test['sentences'].append(sent)
        test['tags'].append(tags)
        tags = []
        sent = []
        contSents = contSents + 1
    sent.append(row['Word'].lower())
    tags.append(row['Tag'])
test['sentences'].append(sent)
test['tags'].append(tags)

In [13]:
len(test['sentences'])

292

In [14]:
transformer = 'bert-base-multilingual-uncased'

In [15]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

In [16]:
from NERDA.models import NERDA
model = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

2021-12-21 09:00:39.871388: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-21 09:00:39.871437: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Device automatically set to: cpu


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
model.train()


 Epoch 1 / 40


 44%|█████████████████████████████████████████████████████████████████████████████▎                                                                                                | 4/9 [00:08<00:10,  2.13s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 196 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:17<00:00,  1.94s/it]


Train Loss = 0.5945828889186183 Valid Loss = 0.3404291868209839

 Epoch 2 / 40


 44%|█████████████████████████████████████████████████████████████████████████████▎                                                                                                | 4/9 [00:08<00:10,  2.09s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 196 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:17<00:00,  1.91s/it]


Train Loss = 0.2811326247950395 Valid Loss = 0.20301101273960537

 Epoch 3 / 40


...

 44%|█████████████████████████████████████████████████████████████████████████████▎                                                                                                | 4/9 [00:08<00:10,  2.05s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 196 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:17<00:00,  1.92s/it]


Train Loss = 0.0004222291878098379 Valid Loss = 0.376625152097808

 Epoch 40 / 40


 44%|█████████████████████████████████████████████████████████████████████████████▎                                                                                                | 4/9 [00:08<00:10,  2.04s/it]/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 196 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:17<00:00,  1.93s/it]

Train Loss = 0.0005649104083052711 Valid Loss = 0.3766310645474328


'Model trained successfully'

In [18]:
model.evaluate_performance(test)

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #214 length 145 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 1 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 1 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'

,Level,F1-Score,Precision,Recall
0,B-asp,0.779319,0.784264,0.774436
1,I-asp,0.403101,0.684211,0.285714
0,AVG_MICRO,0.726681,NaN,NaN
0,AVG_MICRO,0.591210,NaN,NaN


In [19]:
y_pred = model.predict(test['sentences'])

/dados/usuarios/mateus/projetos/absa/16_Doutorado_Implicit/doutorado16/lib/python3.9/site-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #214 length 145 exceeds max_len 128 and has been truncated
  warnings.warn(msg)


In [32]:
aspects = result2aspecList(y_pred, test['sentences'])
res = calculaMetricas(
    aspects, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, 
    output=False
)
res

xml = dicttoxml(res, attr_type=False, custom_root='resultados')
with open('../resultados/bert_multi_indomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    

## Bert Portuguese: Bertimbau

In [34]:
transformer = 'neuralmind/bert-base-portuguese-cased'

# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
    'epochs' : 40,
    'warmup_steps' : 500,                                                   
    'train_batch_size': 13,
    'learning_rate': 0.0001
}

from NERDA.models import NERDA
model2 = NERDA(
    dataset_training = training,
    dataset_validation = validation,
    tag_scheme = tag_scheme, 
    tag_outside = 'O',
    transformer = transformer,
    dropout = dropout,
    hyperparameters = training_hyperparameters
)

Device automatically set to: cpu


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
model2.train()


 Epoch 1 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:36<00:00,  4.03s/it]


Train Loss = 0.6487443366398414 Valid Loss = 0.35070474445819855

 Epoch 2 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:32<00:00,  3.63s/it]


Train Loss = 0.3573871972039342 Valid Loss = 0.3462982773780823

 Epoch 3 / 40


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:17<00:00,  1.90s/it]


Train Loss = 0.0005059979390959294 Valid Loss = 0.33618653493209016

 Epoch 40 / 40


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:17<00:00,  1.89s/it]


Train Loss = 0.0006784177727089021 Valid Loss = 0.33624322005198337


'Model trained successfully'

In [36]:
model2.evaluate_performance(test)

,Level,F1-Score,Precision,Recall
0,B-asp,0.771429,0.800539,0.744361
1,I-asp,0.436620,0.607843,0.340659
0,AVG_MICRO,0.719298,NaN,NaN
0,AVG_MICRO,0.604024,NaN,NaN


In [37]:
y_pred = model2.predict(test['sentences'])

In [38]:
aspects = result2aspecList(y_pred, test['sentences'])
res = calculaMetricas(
    aspects, 
    aspectsTest['datasets'][domain], 
    tipos,
    lemma=False, 
    output=False
)
xml = dicttoxml(res, attr_type=False, custom_root='resultados')
with open('../resultados/bert_bertimbau_indomain_'+domain+'.xml', 'w') as file:
    dom = parseString(xml)
    file.write(dom.toprettyxml())    
res

{'totalAspects': 518,
 'pred': 122,
 'tp': 361,
 'fp': 41,
 'fn': 157,
 'precisao': 0.8980099502487562,
 'precisaoLaplace': 0.8960396039603961,
 'revocacao': 0.696911196911197,
 'medidaf': 0.7847826086956523,
 'totalExplicitos': 292,
 'explicitosOk': 36,
 'pExplicitos': 0.1232876712328767,
 'explicitsList': ['conforto',
  'localização',
  'carpete',
  'portaria',
  'piscina',
  'limpeza',
  'chuveiro',
  'elevador',
  'frigobar',
  'aeroporto',
  'funcionários',
  'porteiro',
  'cozinha',
  'shopping',
  'cidade',
  'funcionário',
  'garagem',
  'aparência',
  'rodoviária',
  'estacionamento',
  'serviço',
  'rua',
  'hotel',
  'telefone',
  'pia',
  'padrão',
  'recepção',
  'atendimento',
  'instalações',
  'preço',
  'quarto',
  'tv',
  'lavanderia',
  'estabelecimento',
  'apartamento',
  'lojas'],
 'totalImplicitos': 226,
 'implicitosOk': 57,
 'pImplicitos': 0.252212389380531,
 'implicitsList': ['cheiro',
  'no centro',
  'valor',
  'banheiros',
  'limpas',
  'sujeira',
  'localiz